In [123]:
import pandas as pd
import requests
import datetime as dt
import json
import sys
import os

In [156]:
ebird_token = 'j6c7l80ga2ib'

In [169]:
##get latest trailhead hotspot dataset
##TODO: #19 hardcode is an anti-pattern, fix to fetch and load the latest file
trailheadHotspots = pd.DataFrame()
trailheadHotspots = pd.read_csv('https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadHotspots/trailheadHotspots_Y2022_M1_D28.csv')
trailheadHotspots.head(5)

,locId,locName,countryCode,subnational1Code,subnational2Code,lat,lng,latestObsDt,numSpeciesAllTime,StopName
0,L10128988,Berntsen Park,US,US-WA,US-WA-033,47.538439,-122.043281,2021-12-15 11:46,15.0,EastSunsetWay
1,L10129014,"Confluence Park, Issaquah",US,US-WA,US-WA-033,47.536469,-122.039342,2022-01-08 12:54,44.0,EastSunsetWay
2,L8365620,Issaquah High School,US,US-WA,US-WA-033,47.522490,-122.028687,2022-01-25 08:00,58.0,EastSunsetWay
3,L8102503,Issaquah Salmon Hatchery,US,US-WA,US-WA-033,47.529503,-122.039512,2022-01-21 14:43,35.0,EastSunsetWay
4,L3352189,"Maple Street ponds, Issaquah",US,US-WA,US-WA-033,47.543027,-122.053715,2021-11-30 10:12,46.0,EastSunsetWay


In [170]:
trailheadHotspots.value_counts('subnational2Code')

subnational2Code
US-WA-033    99
US-WA-035    12
dtype: int64

Duplicate Check

Duplicate checking is done here rather than in the hotspot factory to prevent shelling of the eBird API while still allowing trailheads in close proximity to each other to pull the same observation data. 

In [134]:
##TODO #21 move this section down between the recency checker and observation fetching service
##pull in trailhead reference as base for duplicate hotspot removal

trailheadRef = pd.DataFrame()
trailheadRef_json = 'https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadsRef.json'
response = requests.get(trailheadRef_json)
trailheadRef = pd.DataFrame(response.json())

dfk = pd.merge(
    left=trailheadHotspots,
    right=trailheadRef,
    how='left',
    left_on='StopName',
    right_on='StopName')

,Route,StopName,Address,Latitude,Longitude
0,IssaquahAlps,EastSunsetWay,"661-831 E Sunset Way, Issaquah, WA 98027",47.529635,-122.025119
1,IssaquahAlps,HighSchool,"Parking lot, The Rainier Trail, Issaquah, WA 9...",47.519345,-122.029801
2,IssaquahAlps,ChiricoTrail_PooPooPoint,"11400 Issaquah-Hobart Road Southeast, Issaquah...",47.499949,-122.021730
3,IssaquahAlps,SquakMountain,"13201 Squak Mountain Rd SE, Issaquah, WA 98027",47.481465,-122.053997
4,IssaquahAlps,NullMountain,None,NaN,NaN


##calculate absolute distance from stop location
##why did I do this, I don't need relevancy
for i in dfk.itertuples():
    latInt = dfk['lat']
    latitudeInt = dfk['Latitude']
    lngInt = dfk['lng']
    longitudeInt = dfk['Longitude']
    dfk['dist_fromTH'] = abs(latitudeInt-latInt) + abs(longitudeInt-lngInt)
dfk['dist_fromTH'].astype('int32')

In [141]:
##TRASH COLLECTION, remove and/or refactor as part of issue #21

##sort to push to the hotspot closest to the trailhead to the top of the subset
dfk.sort_values(by=['locName','dist_fromTH'],inplace=True)
##remove any duplicate hotspots except for the hotspot closest to the trailhead
dfk_cooked = dfk.drop_duplicates(subset=['locName'],keep='first')
dfk_cooked.shape

(56, 15)

In [8]:
##TODO #7: write engine to check for new hotspots; also supports addition of new stops

In [139]:
##TRASH COLLECTION, remove as part of issue #21
THhotspots_cooked = dfk_cooked[['locId','locName','StopName']]

##Checking for new hotspots and observations
1. Compare the latest trailheadHotspots eBird locId's with the eBird locId's in the cooking DB
2. For new locId's:
    1. Fetch historical data from the beginning of 2019 to today()-1 from the eBird REST API
    2. Concat and export the results as a .csv
3. For locId's with observations in the DB, obtain the most recent observation date from the eBird REST API for that hotspot:
    1. Make a list of locId's WHERE eBird's latest observation date is more recent than the latest observation date in the DB. Using the most recent DB obsDt as a start date and the most recent eBird API obsDt as an end date, fetch historical sightings from the eBird service
    2. IF the latest obsDt for a locId/hotspot from eBird is within 15 days of the latest obsDt in the DB, remove the locId from the list of locId's to be passed into the historical eBird sighting service
        - TODO: #20 write 'aging' and 'decayed' testing, error logging

In [85]:
##obtain list of files in the 'cooking' DB
fileList = pd.DataFrame()
githubURL = 'https://api.github.com/repos/Tanag3r/trailheadDirect_birds/contents/cooking'
hdr = {'accept':'application/vnd.github.v3+json'}
pyld = {}
req = requests.get(githubURL,headers=hdr,data=pyld)
fileListJSON = pd.DataFrame(req.json())
fileList = fileList.append(fileListJSON[['name','download_url','git_url']])


,name,download_url,git_url,tempName
0,ChiricoTrail_PooPooPoint_Obs_2019to2021.csv,https://raw.githubusercontent.com/Tanag3r/trai...,https://api.github.com/repos/Tanag3r/trailhead...,0 ChiricoTrail_PooPooPoint_Obs_2019to2021.c...
1,EastSunsetWay_Obs_2019to2021.csv,https://raw.githubusercontent.com/Tanag3r/trai...,https://api.github.com/repos/Tanag3r/trailhead...,0 ChiricoTrail_PooPooPoint_Obs_2019to2021.c...
2,HighSchool_Obs_2019to2021.csv,https://raw.githubusercontent.com/Tanag3r/trai...,https://api.github.com/repos/Tanag3r/trailhead...,0 ChiricoTrail_PooPooPoint_Obs_2019to2021.c...
3,MargaretsWay_Obs_2019to2021.csv,https://raw.githubusercontent.com/Tanag3r/trai...,https://api.github.com/repos/Tanag3r/trailhead...,0 ChiricoTrail_PooPooPoint_Obs_2019to2021.c...
4,MountSi_Obs_2019to2021.csv,https://raw.githubusercontent.com/Tanag3r/trai...,https://api.github.com/repos/Tanag3r/trailhead...,0 ChiricoTrail_PooPooPoint_Obs_2019to2021.c...


In [142]:
##get the latest observation date for each trailhead from the observation csv's in the 'cooking' DB
mostRecentObs = []
for name in fileList.itertuples():
    nme = name.name
    flnme = nme[:6]
    ##then read in the csv, reusing that name
    flnme = pd.read_csv(name.download_url)
    flnme.sort_values(by=['locId','obsDt'],ascending=True,inplace=True)
    flnme.drop_duplicates(subset=['locId'],keep='first',inplace=True)
    funk = flnme[['locId','obsDt']]
    mostRecentObs.append(funk)
obslist = pd.concat(mostRecentObs,ignore_index=True)

obslist.sort_values(by=['locId','obsDt'],ascending=False,inplace=True,ignore_index=True)
obslist.drop_duplicates(subset=['locId'],keep='first',inplace=True,ignore_index=True)
obslist = pd.DataFrame(obslist[['locId','obsDt']])
obslist['obsDt'] = obslist['obsDt'].astype('datetime64[ns]')


In [165]:
##get the latest observation date for each locId/hotspot
##22: update the mess in the two lines below to get only subnational2code from trailheadHotspots
THlocIds = trailheadHotspots[['subnational2Code','locId']].sort_values(by=['subnational2Code','locId'],ascending=False,ignore_index=True)
THlocIds.drop_duplicates(subset=['locId'],keep='first',inplace=True)
ebirdLocs = []
##22: update loop below with fetches of https://api.ebird.org/v2/ref/hotspot/info/{{subnational2Code}}
    ##result contains the latest sighting date at each location
for locId in THlocIds.itertuples():
    time.sleep(0.3)
    ebird_baseUrl = 'https://api.ebird.org/v2/ref/hotspot/info/'
    ebird_url = ebird_baseUrl + locId.locId
    ebird_auth = {'X-eBirdApiToken': ebird_token}
    ebird_params = {
        'back':'30',
        'fmt':'json'
        }
    ebird_request = requests.get(ebird_url,params=ebird_params,headers=ebird_auth)
    if ebird_request.status_code == requests.codes.ok:
        ebird_resp = pd.DataFrame(ebird_request.json(),index=[0])
        ebirdLocs.append(ebird_resp)
    ebird_request.raise_for_status()

ebirdLocs_toDate = pd.concat(ebirdLocs)

In [167]:
##TODO #22 recency refactor: get the latest date some other way; eBird hotspot info does not contain the latest date
ebirdLocs_toDate.dtypes
##TODO #23 recency refactor: pare down results of hotspot batches to match known Trailhead Direct eBird hotspots

locId                object
name                 object
latitude            float64
longitude           float64
countryCode          object
countryName          object
subnational1Name     object
subnational1Code     object
subnational2Code     object
subnational2Name     object
isHotspot              bool
locName              object
lat                 float64
lng                 float64
hierarchicalName     object
locID                object
dtype: object

In [29]:
##todo #8: write recency check engine and updater. --DONE
##TODO: #15 Check for a gap between the latest run time and current date



today = dt.date.today()
dateList_hist = []
for x in pd.date_range(start='1/1/2021',end=today,freq='D'):
    dateList_hist.append(x)

In [24]:
import time

ObsHist = []

for locId in xCookedHotspots.itertuples():
    for date in dateList_hist:
        time.sleep(0.5)
        date = date
        ymd = '{}/{}/{}'.format(date.year,date.month,date.day)
        ebird_baseUrl = 'https://api.ebird.org/v2/data/obs/'
        ebird_url = ebird_baseUrl + locId.locId + '/historic/' + ymd
        ebird_auth = {'X-eBirdApiToken': ebird_token}
        ebird_params = {
            'fmt':'json',
            'detail':'simple'
        }
        ebird_request = requests.get(ebird_url,params=ebird_params,headers=ebird_auth)
        if ebird_request.status_code == requests.codes.ok:
            ebird_resp = pd.DataFrame(ebird_request.json())
            ObsHist.append(ebird_resp)
        ebird_request.raise_for_status()

In [25]:
batch_ObsHist = pd.concat(ObsHist)
batch_ObsHist.head()

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
0,commer,Common Merganser,Mergus merganser,L4381196,Cedar Grove Natual Area,2020-04-26 13:30,1,47.462973,-122.080936,True,False,False,S67866709
1,baleag,Bald Eagle,Haliaeetus leucocephalus,L4381196,Cedar Grove Natual Area,2020-04-26 13:30,2,47.462973,-122.080936,True,False,False,S67866709
2,rebsap,Red-breasted Sapsucker,Sphyrapicus ruber,L4381196,Cedar Grove Natual Area,2020-04-26 13:30,2,47.462973,-122.080936,True,False,False,S67866709
3,amerob,American Robin,Turdus migratorius,L4381196,Cedar Grove Natual Area,2020-04-26 13:30,2,47.462973,-122.080936,True,False,False,S67866709
0,rocpig,Rock Pigeon,Columba livia,L4381196,Cedar Grove Natual Area,2020-04-28 12:18,3,47.462973,-122.080936,True,False,False,S70289791


In [26]:
##TODO #16 dynamically name files to denote recency
batch_ObsHist.to_csv('MountSi_Obs_2019to2021.csv',sep=',',index=False)